<H2>CHAPTER 5 - Lab: Cross-Validation and the Bootstrap</H2>

> This section explores the resampling techniques covered in this chapter. Some of the commands may take a considerable amount of time to run on your computer. We begin by organizing most of the required imports at the top level.


In [3]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
summarize,
poly)
from sklearn.model_selection import train_test_split


There are several new imports needed for this lab.

In [5]:
 from functools import partial
from sklearn.model_selection import \
(cross_validate,
KFold,
ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

<h4>The Validation Set Approach</h4>

The dataset is divided into training and validation subsets using the train_test_split() function. Since the dataset contains 392 observations, it is partitioned into two equally sized sets of 196 samples by specifying test_size=196. To ensure that the results are fully reproducible, it is good practice to fix the random seed when performing randomized operations. In this case, reproducibility is achieved by setting the random_state parameter to 0.

In [8]:
Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto,
test_size=196,
random_state=0)

Now we can fit a linear regression using only the observations corresponding to the training set Auto_train.

In [10]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()

The predict() method is applied to the model results using the model matrix constructed from the validation dataset. The validation mean squared error (MSE) is then computed.

In [13]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred)**2)

np.float64(23.616617069669882)

> Consequently, the estimated validation MSE for the linear regression model is 23.62. The validation error can also be evaluated for polynomial regression models of higher degree. To facilitate this, we first define a function, `evalMSE()`, which takes a model specification along with training and test datasets and returns the mean squared error computed on the test set.


In [17]:
def evalMSE(terms,
            response,
            train,
            test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]
    X_test = mm.transform(test)
    y_test = test[response]
    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)
    return np.mean((y_test - test_pred)**2)


This function is then used to estimate the validation MSE for linear, quadratic, and cubic models. The enumerate() function is employed to iterate through the models while simultaneously providing both the index and the corresponding value at each step.

In [20]:
# Initialize an array to store MSE for degrees 1, 2, 3
MSE = np.zeros(3)

# Loop through polynomial degrees 1, 2, 3
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                        'mpg',
                        Auto_train,
                        Auto_valid)

# Display the MSE array
MSE


array([23.61661707, 18.76303135, 18.79694163])

The resulting error rates are 23.62, 18.76, and 18.80, respectively. However, if a different training–validation split is selected instead…

In [23]:
# Split dataset into training and validation sets
Auto_train, Auto_valid = train_test_split(Auto,
                                          test_size=196,
                                          random_state=3)

# Initialize array to store MSE for polynomial degrees 1, 2, 3
MSE = np.zeros(3)

# Loop over polynomial degrees
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                        'mpg',
                        Auto_train,
                        Auto_valid)

# Display the MSE array
MSE


array([20.75540796, 16.94510676, 16.97437833])

The sklearn_sm() class takes a statsmodels model as its first argument. It also accepts two optional parameters: model_str, which is used to define a model formula, and model_args, a dictionary containing additional arguments required during model fitting. For instance, when fitting a logistic regression model, the family parameter must be specified and is provided through model_args = {'family': sm.families.Binomial()}.

In [25]:
 hp_model = sklearn_sm(sm.OLS,
MS(['horsepower']))
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model,
X,
Y,
cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err

np.float64(24.231513517929233)

> The `cross_validate()` function takes as input an object that implements the `fit()`, `predict()`, and `score()` methods, along with a feature matrix (X) and a response variable (Y). An additional argument, `cv`, is used to specify the cross-validation strategy: providing an integer (K) results in (K)-fold cross-validation. In this case, the value is set equal to the total number of observations, yielding leave-one-out cross-validation (LOOCV)
> The `cross_validate()` function returns a dictionary containing several outputs; here, we focus on the cross-validated test score, namely the mean squared error (MSE), which is estimated to be 24.
>
> This procedure can be extended to polynomial regression models of increasing complexity. To automate the process, a for loop is used to fit polynomial models of degrees 1 through 5, compute the corresponding cross-validation errors, and store them in the (i)th element of the vector `cv_error`. The loop variable `d` denotes the degree of the polynomial. The vector is initialized prior to the loop, and the computation may take a few seconds to complete.


In [29]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
Y = np.array(Auto['mpg'])
M = sklearn_sm(sm.OLS)

for i, d in enumerate(range(1, 6)):
    # Design matrix for polynomial regression
    X = np.column_stack([H**p for p in range(d+1)])
    
    # LOOCV with MSE
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=Auto.shape[0],                # LOOCV
                          scoring='neg_mean_squared_error',
                          return_train_score=False)
    
    # Store MSE (convert negative to positive)
    cv_error[i] = -np.mean(M_CV['test_score'])

cv_error


array([24.23151352, 19.24821312, 19.33498406, 19.4244303 , 19.0332262 ])

Earlier, we introduced the outer() method in combination with np.power(). The outer() method can be applied to any function that takes two arguments, such as add(), min(), or power(). It takes two arrays as input and produces a larger array in which the specified operation is applied to every possible pair of elements from the two arrays.

In [31]:
 A = np.array([3, 5, 9])
B = np.array([2, 4])
np.add.outer(A, B)

array([[ 5,  7],
       [ 7,  9],
       [11, 13]])

> In the CV example above, we set (K = n), but it is also possible to choose (K < n). The code is very similar to the previous example and runs significantly faster. Here, we use `KFold()` to divide the data into (K = 10) random folds. A random seed is set via `random_state` for reproducibility, and a vector `cv_error` is initialized to store the cross-validation errors for polynomial fits of degrees one through five.


In [34]:
cv_error = np.zeros(5)

# Use the same K-Fold splits for all degrees
cv = KFold(n_splits=10, shuffle=True, random_state=0)

for i, d in enumerate(range(1, 6)):
    # Create polynomial design matrix for degree d
    X = np.power.outer(H, np.arange(d+1))
    
    # Perform cross-validation
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=cv)
    
    # Store mean test score
    cv_error[i] = np.mean(M_CV['test_score'])

# Display cross-validation errors
cv_error


array([24.20766449, 19.18533142, 19.27626666, 19.47848402, 19.13719075])

> The `cross_validate()` function is flexible and allows different data-splitting strategies. For example, it can be used to implement the validation set approach just as easily as K-fold cross-validation.


In [36]:
validation = ShuffleSplit(n_splits=1,
test_size=196,
random_state=0)
results = cross_validate(hp_model,
Auto.drop(['mpg'], axis=1),
Auto['mpg'],
cv=validation);
results['test_score']

array([23.61661707])

The variability in the test error can be estimated by executing this code:

In [38]:
validation = ShuffleSplit(n_splits=10,
test_size=196,
random_state=0)
results = cross_validate(hp_model,
Auto.drop(['mpg'], axis=1),
Auto['mpg'],
cv=validation)
results['test_score'].mean(), results['test_score'].std()

(np.float64(23.802232661034164), np.float64(1.4218450941091862))

<h4>Estimation of the accuracy of statistic of interest</h4>

> A function `alpha_func()` is created, which takes as input a DataFrame `D` containing columns `X` and `Y`, along with a vector `idx` specifying the observations used to estimate (\alpha). The function then returns the estimate of (\alpha) based on the selected observations.


In [42]:
Portfolio = load_data('Portfolio')

def alpha_func(D, idx):
    cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
    return ( (cov_[1,1] - cov_[0,1]) / (cov_[0,0] + cov_[1,1] - 2*cov_[0,1]) )


> This function computes an estimate of (\alpha) by applying the minimum variance formula (5.7) to the observations specified by the `idx` argument. For example, the command below estimates (\alpha) using all 100 observations.


In [44]:
alpha_func(Portfolio, range(100))

np.float64(0.57583207459283)

> Next, we randomly select 100 observations from `range(100)` with replacement. This effectively creates a new bootstrap dataset and allows us to recompute (\hat{\alpha}) based on the resampled data.


In [46]:
 rng = np.random.default_rng(0)
alpha_func(Portfolio,
rng.choice(100,
100,
replace=True))

np.float64(0.6074452469619004)

> This method can be extended to define a simple function, `boot_SE()`, which computes the bootstrap standard error for any function that takes a DataFrame as input.


In [50]:
def boot_SE(func,
            D,
            n=None,
            B=1000,
            seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index,
                         n,
                         replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2)


> Note the use of `_` as the loop variable in `for _ in range(B)`. This is commonly done when the loop counter itself is not needed and the purpose is simply to execute the loop `B` times. We can now use our function to assess the accuracy of our (\alpha) estimate with `B = 1,000` bootstrap replications.


In [52]:
alpha_SE = boot_SE(alpha_func,
Portfolio,
B=1000,
seed=0)
alpha_SE

np.float64(0.09118176521277699)

<h4>Estimating the Accuracy of a Linear Regression Model</h4>

> We begin by defining a generic function `boot_OLS()` for bootstrapping a regression model, where the regression is specified using a formula. The `clone()` function is used to create a copy of the formula that can be refit to the resampled DataFrame. This ensures that any derived features, such as those created with `poly()` (which will be demonstrated shortly), are recalculated on the bootstrap sample.


In [89]:
def boot_OLS(model_matrix, response, D, idx):
    D_ = D.iloc[idx]  # Use iloc to index by integer positions
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

In [91]:
hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')

In [92]:
rng = np.random.default_rng(0)

np.array([
    hp_func(Auto, rng.choice(Auto.shape[0], Auto.shape[0], replace=True))
    for _ in range(10)
])


array([[39.88064456, -0.1567849 ],
       [38.73298691, -0.14699495],
       [38.31734657, -0.14442683],
       [39.91446826, -0.15782234],
       [39.43349349, -0.15072702],
       [40.36629857, -0.15912217],
       [39.62334517, -0.15449117],
       [39.0580588 , -0.14952908],
       [38.66688437, -0.14521037],
       [39.64280792, -0.15555698]])

Then w We apply the `boot_SE()` function to calculate the standard errors of 1,000 bootstrap estimates for both the intercept and slope coefficients.


In [72]:
hp_se = boot_SE(hp_func,
Auto,
B=1000,
seed=10)
hp_se

intercept     0.731176
horsepower    0.006092
dtype: float64



This shows that the bootstrap estimate of SE(β̂₀) is 0.85, while the bootstrap estimate of SE(β̂₁) is 0.0074. As explained in Section 3.1.2, standard formulas can also be used to calculate the standard errors of the regression coefficients in a linear model. These values can be obtained using the `summarize()` function from **ISLP.smo that?


In [74]:
hp_model.fit(Auto, Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se

intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

Here b

Below, we calculate both the bootstrap standard error estimates and the standard linear regression estimates for the quadratic model fitted to the data. Because this model fits the data well (Figure 3.8), the bootstrap estimates now align more closely with the standard estimates of SE(β̂₀), SE(β̂₁), and SE(β̂o that?


In [77]:
quad_model = MS([poly('horsepower', 2, raw=True)])
quad_func = partial(boot_OLS,
quad_model,
'mpg')
boot_SE(quad_func, Auto, B=1000)

intercept                                  1.538641
poly(horsepower, degree=2, raw=True)[0]    0.024696
poly(horsepower, degree=2, raw=True)[1]    0.000090
dtype: float64

We compare these results with the standard errors obtained from sm.OLS().

In [78]:
M = sm.OLS(Auto['mpg'],
quad_model.fit_transform(Auto))
summarize(M.fit())['std err']

intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64